In [1]:
from datasets.datamodule import DEEPscreenDataModule
from engine.system import DEEPScreenClassifier
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
import pandas as pd

/home/sjinich/disco/che_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("../../.data/processed/CHEMBL2581.csv")

In [3]:
data

,comp_id,CHEMBL2581,data_split,smiles
0,CHEMBL104783,1,validation,COc1ccc(CN[C@@H](C(=O)N[C@H](C(=O)NCc2ccccc2)C...
1,CHEMBL104966,1,validation,CC(C)[C@H](NC(=O)[C@H](NCc1ccccc1)[C@H](O)[C@H...
2,CHEMBL1076901,1,train,CC(C)[C@H](NC(=O)[C@H](C)C[C@H](O)[C@H](COCc1c...
3,CHEMBL1076902,1,train,CC(C)[C@H](NC(=O)[C@H](C)C[C@H](O)[C@H](COCc1c...
4,CHEMBL1076905,1,train,CC(C)[C@H](NC(=O)[C@H](C)C[C@H](O)[C@H](COc1cc...
...,...,...,...,...
2265,CHEMBL96943,1,validation,C=CC1C=CC=CC1/C=C/OCC(=O)NC(C(=O)NC(Cc1ccccc1)...
2266,CHEMBL97013,1,train,CCCCNC(=O)CC(O)C(CC(C)C)NC(=O)C(NC(=O)Cc1ccc2c...
2267,CHEMBL97072,1,validation,C=Cc1ccccc1/C=C/OCC(=O)NC(C(=O)NC(CC(C)C)C(O)C...
2268,CHEMBL97805,1,validation,CCCCNC(=O)CC(O)C(CC(C)C)NC(=O)C(NC(=O)Cc1cc(OC...


In [4]:
%load_ext tensorboard

In [5]:
trainer = Trainer(max_epochs=100)
model = DEEPScreenClassifier(fully_layer_1=256,fully_layer_2=32,drop_rate=0.5,learning_rate=0.0001,batch_size=32,experiment_result_path="../../.experiments/chembl2581")
datamodule = DEEPscreenDataModule(data=data,target_id="CHEMBL2581",batch_size=32,experiment_result_path="../../.experiments/chembl2581",data_split_mode="non_random_split",tmp_imgs=True)
trainer.fit(model,datamodule=datamodule)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO: Using hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.5), ('learning_rate', 0.0001), ('batch_size', 32), ('experiment_result_path', '../../.experiments/chembl2581')]
ERROR: invalid columns of df
CRITICAL: Dataframe not valid dataframe must contain the following columns{'comp_id','smiles','bioactivity'}


InvalidDataframeException: dataframe must contain the following columns{'comp_id','smiles','bioactivity'}